# FIT5196 Task 2 in Assessment 1
#### Student Name: Yicheng Zhang
#### Student ID: 27699641

Date: 20/03/2018

Version: 1.0

Environment: Python 3.6.3 and Jupyter notebook

Libraries used: 
* pandas (for dataframe, included in Anaconda Python 3.6) 
* re (for regular expression, included in Anaconda Python 3.6) 
* numpy (for numpy array, included in Anaconda Python 3.6) 
* pdftables(for parse pdf file, included pdftable.six in third party package)


## 1.  Import libraries

In [ ]:
import sys
from pdftables import get_tables
import pandas as pd
import numpy as np
import re
sys.version

## 2. Parse PDF File

### 2.1. Load PDF File
Unlike the excel files, pdf files usually not well-structured, we need to use some third-party packages in python to Extract the data from the pdf file.
In this case, we choose `pdftables`

`pdftables` create list objects for each page store the records also as list object.

`get_tables` is a method which can do the task specifically

** Create Pdf file object, load the tables in the pdf using `get_tables` method **

In [ ]:
pdfFile = 'health.pdf'
pdfobj = open(pdfFile, 'rb') #read pdf as object
tables = get_tables(pdfobj) #extract tables from pdf file

** Check the content in the tables **

In [ ]:
for table in tables:
    for row in table[:10]:
        print (row)
    print ('==========================\n') #dsiplay the results extract from pdf

** Check the content in the tables, for each page start with 5th row(Countries)**

In [ ]:
for table in tables:  #4 pages
    for row in table[5:]:  # 
        if 'THE STATE' in row[0]:
            break
        if row[0] == 'SUMMARY':
            print (row)
            break
        print (row)
    print ('==========================\n') #dsiplay desired results extract from pdf

** Check number of records in each page**

In [ ]:
print(len(tables[0]))
print(len(tables[1]))
print(len(tables[2]))
print(len(tables[3])) #check the records from each tables(pages)

### 2.2. Processing Data 
After obeserving the above results, some finding can be summarized:
* It can be found that for first three pages, the effective data always begin in 5 element, end with total numbers minus 2 element(The last two records are empty)

* For the last page, there are only 14 countries

** Update the pages, create list object copy in case errors**



** Update the pages, create list object in case errors**

In [ ]:
tables[0] = tables[0][5:67] #First page Afghanistan to Fiji len-2 for Fiji
tables[1] = tables[1][5:68] #Second page Finland to Nepal len-2 for Nepal
tables[2] = tables[2][5:68] #Third page Netherland to Tuvalu len-2 for Tuvalu
tables[3] = tables[3][5:19] #Fourth page Uganda to Zimbabwe 14 countries in final page
page1 = tables[0].copy()
page2 = tables[1].copy()
page3 = tables[2].copy()
page4 = tables[3].copy() #process the data in copy version in case errors

** Check the content in each page**

In [ ]:
page1

In [ ]:
page2

In [ ]:
page3

In [ ]:
page4

There are still many problems in the data using the data in page1,3 as example:
```python
['Afghanistan','63','89','53','3956','33','74','73','65','60','62',...]
```
the data in the position(index) 4 which is `3956` is not correct when we check the original pdf file. This data should be divded by `39` and `56`
The reason for this, may caused by pdftables itself.
- - -
Similarily, this situation occur in page2 the position(index) 5
```python
['Finland', '100', '100', '100', '99','9999', '–', '99', '92',...]
```
- - -
We can use the regular expression to split two numbers with 2 digits easily, it is nearly impossible to split a 1-digit number between two numbers, after we split by regular expression, we should using some method to check the result.

For the above splits, index from 4-6, refer to the pdf file, it represent the `Use of basic sanitation services (%)` in 2015 with `total` `urban` `rural`, it can be known by common sense that `total` should between `urban` and `rural`, if the condition cannot be satisfied, there are high chance there may be some mistake, for instance, first is 1-digit number, second one have 2-digits.

For the 4th page on the other hand, there are many merged mistakes in data.
```python
['Uganda397332','192817938978','82','82','0','78','78','07887','80','4781','62','78']
```
the data in position(index) 0,1,7,9 should respectively be:
* `Uganda397332` to `Uganda`,`39`,`73`,`32`
* `192817938978` to `19`,`28`,`17`,`93`,`89`,`78`
* `07887` to `0`,`78`,`87`
* `4781` to `47`,`81`
- - -
Using the regular expression can do this split job, however, there are some potential problem:
1. There are some special charcter in the data, such as x,-
2. Observing the data from the original file, the data are all percentage numbers, which means, the data range should inside 0 - 100,we can use a regular expression to match these numbers apply with for loop.
3. After spliting, the results should be checked.
```python
['Venezuela (Bolivarian Republic of)979986','959872999884','82','88','53','84','84','47775','72x','38x–','–','–']
```

In this data, in index 7, the string is `47775`, the expected output should be `47` `7` `75`,if we use the regular expression to find all the number between 0-100, the output will be `47` `77` `5` which is incorrect.

check the `75`, it refer `Protection at birth (PAB) against tetanus`, look at the references[1], in 2016, this data in the world should larger than 60%, because this data is the last one when split, if this number smaller than 10, it can be nearly assert there is a mistake in this record.

** Using the regular expression to split and fill the records **

In [ ]:
for index in range(len(page1)):
    split = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page1[index].pop(4)) #split merged data in index 4
    page1[index] = page1[index][:4] + split + page1[index][4:]
page1

** Check the potential error in page1**

In [ ]:
for record in page1:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6]): #when total bigger than both urban and rural
                print(record)     #check potential error

It can be found that in the record with index4 and index5, error may exist, cause the `71` is a number represent total, which is not between `8` and `4`(`urban` and `rural`),this situation can be caused by the second number in index4 should be first number in index5

** Try to fix the split error in page1**

In [ ]:
for record in page1:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6]): #when total bigger than both urban and rural
                split1 = re.findall(r'(\d)(\d)', record[4]) 
                split2 = re.findall(r'(\d)', record[5])
                record[4] = split1[0][0]       #change index 4 to 1 digit
                record[5] = split1[0][1]+split2[0][0] # change index 5 to 2 digits

** Double Check the split error in page1**

In [ ]:
for record in page1:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6]): #when total bigger than both urban and rural
                print(record)

** split page 2 **

In [ ]:
for index in range(len(page2)):
    split = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page2[index].pop(5)) #split merged data in index 5
    page2[index] = page2[index][:5] + split + page2[index][5:]
page2

** Check the potential error in page2**

In [ ]:
for record in page2:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6]) and len(record[6])<2: #when total bigger than both urban and rural
                print(record)

** split page 3 **

In [ ]:
for index in range(len(page3)):
    split = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page3[index].pop(4))#split merged data in index 4
    page3[index] = page3[index][:4] + split + page3[index][4:]
page3

** Check the potential error in page3**

In [ ]:
for record in page3:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6])and len(record[6])<2: #when total bigger than both urban and rural
                print(record)

** split page 4**

In [ ]:
for index in range(len(page4)):
    name = re.findall(r'[a-zA-Z()\s]+', page4[index][0])
    data_1_3 = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page4[index][0])
    page4[index][0] = name[0]
    page4[index] = page4[index][:1] + data_1_3 + page4[index][1:] 
    data_4_10 = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page4[index].pop(4))
    page4[index] = page4[index][:4] + data_4_10 + page4[index][4:]
    data_15_17 = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page4[index].pop(15))
    page4[index] = page4[index][:15] + data_15_17 + page4[index][15:]
    data_19_20 = re.findall(r'(100+|[1-9][0-9]?x?|[–]|0)', page4[index].pop(19))
    page4[index] = page4[index][:19] + data_19_20 + page4[index][19:]
page4

** Check the potential error in page4 index[0],[1]**

In [ ]:
for record in page4:
     if '–' not in record[5] and '–' not in record[6]:
        if int(record[4]) > int(record[5]) and int(record[4])>int(record[6]): #when total bigger than both urban and rural
                print(record)

In [ ]:
for record in page4:
     if '–' not in record[2] and '–' not in record[3]:
        if int(record[1]) > int(record[2]) and int(record[1])>int(record[3]): #when total bigger than both urban and rural
                print(record)

Merge data in index[7] contain three data, we should check the last data in case there is a 1-digit number in the second data.(accroding to the regluar expression) The last data now in index[17] is `Protection at birth (PAB) against tetanus` and should not less than 60 refer to previous explnation(40 for 2015 data,we use 40 for assert error)

** Check the potential error in page4 index[7]**

In [ ]:
for record in page4:
     if '–' not in record[17]:
        if int(record[17]) < 40: #data from WHO PAB 2015 (assert error if less than 40%)
                print(record)

Merge data in index[9] contain two data, we should check if there are 1-digit data

** Check the potential error in page4 index[9]**

In [ ]:
for record in page4:
     if '–' not in record[18] and '–' not in record[19]:
        if len(record[18]) < 2 or len(record[18])<2:
                print(record)

** Try to fix the error which found in above **

In [ ]:
for record in  page4:
    if '–' not in record[17]:
        if int(record[17]) < 40: #data from WHO PAB
            split1 = re.findall(r'(\d)(\d)', record[16])
            split2 = re.findall(r'(\d)', record[17])
            record[16] = split1[0][0]
            record[17] = split1[0][1]+split2[0][0]
            print(record)

** Double Check the split error in page4 **

In [ ]:
for record in page4:
     if '–' not in record[17]:
        if int(record[17]) < 40: 
                print(record)

** Add all the pages togther, check the records if numbers are correct **

In [ ]:
newpage = page1 + page2 + page3 + page4  # add all list togther
len(newpage)  # check number of records

** Check the data by compare it with original file **

In [ ]:
for row in newpage:
    print (row)
    print ('==========================\n') # check the records

In [ ]:
newpage

** Using the build-in method of `Pandas` named `DataFrame()` create the dataframe by list objects **

In [ ]:
data = pd.DataFrame(newpage) #convert list of lists to dataframe
data

There are some speical charcters in the dataframe `x`
Finding the `x`'s meaning in the original file:
> x Data refer to years or periods other than those specified in the column heading. Such data are not included in the calculation of regional and global averages. Estimates from data years prior to 2000 are not displayed.

It can be found that `x` is a mark for data beside it, may influence the data of global caculations, we only concerned about single countries instead of global statistics, the `x` should be dropped in this case.

the replace method in dataframe also should compare and match `x` in data and replace it, the `regex` variable should be set as 
```python 
True
```

**Replace all the x in the data with empty and check the results**

In [ ]:
data.replace('x','',regex=True,inplace=True) #replace all the x in dataframe
data.head()

Check the dataframe, it can be found that there are some data not numeric `–`.

Finding the `–`'s meaning in the original file:
> – Data not available.

For processing purpose, this symbol should be None value or empty value.

**Replace all the `–` to empty value using `numpy.nan` (Empty value store as float number)**

**Check the results**

In [ ]:
data.replace('–',np.nan,inplace=True) #replace all–in dataframe
data.head()

There are still some problems in the dataframe:
* Columns Names are meaningless
* index should be reset

**Rename the columns, set the first column as the index(which are country names),drop the the first column after setting it as the index of the dataframe**

In [ ]:
data = data.set_index(data[0].values)
data = data.drop(0,axis=1)
data.columns = list(range(len(data.columns)))
data.index.rename('Country Name', inplace=True)  #adjust dataframe similar with excel file
data.head()

### 2.3. Save the Data 
After check the rows and columns in the dataframe, we need to store these data to a CSV file which can be used in the future,`pandas` can use the method named `to_csv` to save the dataframe in the CSV file.

similar with the Excel file, the encoding should also using the `utf-16` for display latin charcter purpose.

As requirement, the data should be integer, the data in the dataframe is float, we should control the number of float digits, which should be 0 after dot in this case. 

In [ ]:
data.to_csv('./health.csv',encoding='utf-16',sep='\t')

## 3. Summary

Based on the above script and process, some conclusions can be drawn:
* Extract data from a PDF file is very hard compare with the structured files, such as excel, csv. However, we can also use the exits package in python for extracting data from PDF. One of the solutions is `pdftables`. It can be found that even using the package, there are still many work to wrangling the data. 
- - -
* Using the regular expression is an effective way to get required results for multiple similar errors in the data wrangling, However, there may exits some cases that cannot use the general term formula, we should double check the data before using the regular expression, ensure it cover all situation.
- - -
* Due to the limitation of the regular expression, data should be checked after using the regular expression. We should check these data by considering the relationship and meaning of data itself, to develop script to check and fix the error.

## 4. References

[pandas](http://pandas.pydata.org/pandas-docs/version/0.15/tutorials.html)
- - -
[pdftables](https://github.com/okfn/pdftables)
_ __
[python data structures](https://docs.python.org/3.6/tutorial/datastructures.html)
- - -
[tutorial on moodle](https://moodle.vle.monash.edu/course/view.php?id=42906&section=8#8)